In [ ]:
%load_ext autoreload
%autoreload 2
from ambiguous.dataset.dataset import *

# EMNIST example

In [ ]:
save_dataset_to_file(dataset_name='EMNIST',
                     og_root='/share/datasets/',
                     new_root='/home/nislah/ambiguous-dataset/ambiguous/data/EMNIST/',
                     blend=0.5) # can specify pairs as argument, otherwise uses default from dataset.py

In [ ]:
trainset = DatasetFromNPY('/home/nislah/ambiguous-dataset/ambiguous/data/EMNIST', train=True)
testset = DatasetFromNPY('/home/nislah/ambiguous-dataset/ambiguous/data/EMNIST', train=False)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=True)
print(len(trainset), len(testset))
x, t = next(iter(trainloader))
i = 0
x.shape, [(t[0][i], t[1][i]) for i in range(5)]

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(make_grid(x, nrow=8).detach().cpu().permute(2,1,0))
plt.axis('off')

# MNIST example

In [ ]:
save_dataset_to_file('MNIST', '/share/datasets/', '/home/nislah/ambiguous-dataset/ambiguous/data/MNIST/',blend=0.5)

In [ ]:
trainset = DatasetFromNPY('/home/nislah/ambiguous-dataset/ambiguous/data/MNIST/', train=True)
testset = DatasetFromNPY('/home/nislah/ambiguous-dataset/ambiguous/data/MNIST/', train=False)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=True)
print(len(trainset), len(testset))
x, t = next(iter(trainloader))
i = 0
x.shape, t[0][i], t[1][i]

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(make_grid(x, nrow=8).detach().cpu().permute(1,2,0))
plt.axis('off')